# Prepare the Data

In [2]:
# Set up files using starter code
import matplotlib.pyplot as plt
import pandas as pd
import scipy.stats as st

In [3]:
# Study data files with starter code
mouse_metadata_path = "data/Mouse_metadata.csv"
study_results_path = "data/Study_results.csv"

In [4]:
# Read the mouse data and study data with starter code
mouse_metadata = pd.read_csv(mouse_metadata_path)
study_results = pd.read_csv(study_results_path)

In [5]:
# Combine the data into a single dataframe
combo_df = pd.merge(mouse_metadata, study_results, how= 'left', on='Mouse ID')

In [6]:
# Display the dataframe
combo_df.head()

,Mouse ID,Drug Regimen,Sex,Age_months,Weight (g),Timepoint,Tumor Volume (mm3),Metastatic Sites
0,k403,Ramicane,Male,21,16,0,45.000000,0
1,k403,Ramicane,Male,21,16,5,38.825898,0
2,k403,Ramicane,Male,21,16,10,35.014271,1
3,k403,Ramicane,Male,21,16,15,34.223992,1
4,k403,Ramicane,Male,21,16,20,32.997729,1


In [7]:
# check for the number of unique mice
unique_mice = combo_df["Mouse ID"].nunique()
unique_mice

249

In [10]:
cleaned_df = combo_df[combo_df.duplicated(subset=["Mouse ID","Timepoint"],keep= False)]
print(cleaned_df)

    Mouse ID Drug Regimen     Sex  Age_months  Weight (g)  Timepoint  \
908     g989     Propriva  Female          21          26          0   
909     g989     Propriva  Female          21          26          0   
910     g989     Propriva  Female          21          26          5   
911     g989     Propriva  Female          21          26          5   
912     g989     Propriva  Female          21          26         10   
913     g989     Propriva  Female          21          26         10   
914     g989     Propriva  Female          21          26         15   
915     g989     Propriva  Female          21          26         15   
916     g989     Propriva  Female          21          26         20   
917     g989     Propriva  Female          21          26         20   

     Tumor Volume (mm3)  Metastatic Sites  
908           45.000000                 0  
909           45.000000                 0  
910           48.786801                 0  
911           47.570392        

In [13]:
unique_cleaned = cleaned_df["Mouse ID"].unique()
unique_cleaned

array(['g989'], dtype=object)

# Generate Summary Statistics

# Create Pie Charts and Bar Charts

# Calculate quartiles, find outliers, and create a box plot

# Create a line plot and a scatter plot

# Calculate correlation and regression

# Submit your final analysis